In [34]:

from skimage.draw import line
from PIL import Image
import numpy as np
import cv2
import pickle

NAIL_DISTANCE = 8

def display_image(image):
    display(Image.fromarray(image))
    
def display_nails(nails, image):
    canvas = np.ones(image.shape, np.uint8) * 255
    for nail in nails:
        canvas[nail[0], nail[1]] = 0
    display_image(canvas)

def display_threads(lines, image):
    canvas = np.ones(image.shape, np.uint8) * 255
    for line in lines:
        canvas = cv2.line(canvas, (line[0][1], line[0][0]), (line[1][1], line[1][0]), 0, 1)
    display_image(canvas)

def generate_nail_positions(dimensions):
    nails = []

    r0, c0 = 0, 0
    r1, c1, = 0, dimensions[1] - 1
    r2, c2 = dimensions[0] - 1, dimensions[1] - 1
    r3, c3 = dimensions[0] - 1, 0

    for i in range(c0, c1, NAIL_DISTANCE):
        nails.append((c0, i))

    for i in range(r1, r2, NAIL_DISTANCE):
        nails.append((i, c1))

    for i in range(c2, c3, -NAIL_DISTANCE):
        nails.append((r2, i))
        
    for i in range(r3, r0, -NAIL_DISTANCE):
        nails.append((i, c3))

    return np.array(nails)


def is_on_same_edge(point1, point2, dimensions):
    if point1[0] == 0 and point2[0] == 0:
        return True
    if point1[0] == dimensions[0] - 1 and point2[0] == dimensions[0] - 1:
        return True
    if point1[1] == 0 and point2[1] == 0:
        return True
    if point1[1] == dimensions[1] - 1 and point2[1] == dimensions[1] - 1:
        return True
    return False

def generate_thread_positions(nails, dimensions):
    threads = []
    for i in range(0, len(nails)):
        for j in range(i, len(nails)):
            if not is_on_same_edge(nails[i], nails[j], dimensions):
                threads.append(np.array([nails[i], nails[j]]))
    return np.array(threads)

def get_other_end_of_thread(thread, nail):
    if np.array_equal(thread[0], nail):
        return thread[1]
    return thread[0]

def get_threads_from_nail(threads, nail):
    threads_from_nail = []
    for thread in threads:
        if np.array_equal(thread[0], nail) or np.array_equal(thread[1], nail):
            threads_from_nail.append(thread)
    return np.array(threads_from_nail)    

def generate_nail_to_threads_dictionary(nails, threads):
    nail_to_thread_dictionary = {}
    it = 0
    for nail in nails:
        if it%10 == 0:
            print("Generating nail to thread dictionary: " + str(it) + "/" + str(len(nails)))
        nail_to_thread_dictionary[nail.tobytes()] = get_threads_from_nail(threads, nail)
        it += 1
    return nail_to_thread_dictionary

def generate_thread_to_points_dictionary(threads):
    thread_to_points_dictionary = {}
    it = 0
    for thread in threads:
        if it%1000 == 0:
            print("Thread " + str(it) + " of " + str(len(threads)))
        it += 1
        a, b = line(thread[0][0], thread[0][1], thread[1][0], thread[1][1])
        points = []
        for i in range(len(a)):
            points.append((a[i], b[i]))
        thread_to_points_dictionary[thread.tobytes()] = np.array(points)
    return thread_to_points_dictionary


# #
# # Test 
# #
# test_image = np.ones((121, 121), np.uint8) * 255

# nails = generate_nail_positions(test_image.shape)
# threads = generate_thread_positions(nails, test_image.shape)

# nail_to_threads_dictionary = generate_nail_to_threads_dictionary(nails, threads)
# thread_to_points_dictionary = generate_thread_to_points_dictionary(threads)

# print("Input image")
# display_image(test_image)

# print("All nails")
# display_nails(nails, test_image)

# print("First nail")
# display_nails([nails[0]], test_image)

# print("Nail from get_other_end_of_thread of first thread")
# display_nails([get_other_end_of_thread(threads[0], nails[0])], test_image)

# print("Threads from get_threads_from_nail")
# display_threads(get_threads_from_nail(threads, nails[0]), test_image)

# print("Threads from dictionary")
# display_threads(nail_to_threads_dictionary[nails[0].tobytes()], test_image)

# print("First thread points from dictionary")
# display_nails(thread_to_points_dictionary[threads[0].tobytes()], test_image)

In [35]:
def pickle_dictionaries(dimensions, filename):

    nails = generate_nail_positions(dimensions)
    threads = generate_thread_positions(nails, dimensions)
    nail_to_threads_dictionary = generate_nail_to_threads_dictionary(nails, threads)
    thread_to_points_dictionary = generate_thread_to_points_dictionary(threads)

    with open("dictionaries/" + filename + "_n.dat", "wb") as n, \
            open("dictionaries/" + filename + "_t.dat", "wb") as t, \
            open("dictionaries/" + filename + "_ntt.dat", "wb") as ntt, \
            open("dictionaries/" + filename + "_ttp.dat", "wb") as ttp:
        pickle.dump(nails, n)
        pickle.dump(threads, t)
        pickle.dump(nail_to_threads_dictionary, ntt)
        pickle.dump(thread_to_points_dictionary, ttp)


 


# pickle_dictionaries((1561, 1561), "1x1")

# pickle_dictionaries((1321, 1761), "3x4")
# pickle_dictionaries((1761, 1321), "4x3")

# pickle_dictionaries((1441, 1801), "4x5")
# pickle_dictionaries((1801, 1441), "5x4")




Generating nail to thread dictionary: 0/810
Generating nail to thread dictionary: 10/810
Generating nail to thread dictionary: 20/810
Generating nail to thread dictionary: 30/810
Generating nail to thread dictionary: 40/810
Generating nail to thread dictionary: 50/810
Generating nail to thread dictionary: 60/810
Generating nail to thread dictionary: 70/810
Generating nail to thread dictionary: 80/810
Generating nail to thread dictionary: 90/810
Generating nail to thread dictionary: 100/810
Generating nail to thread dictionary: 110/810
Generating nail to thread dictionary: 120/810
Generating nail to thread dictionary: 130/810
Generating nail to thread dictionary: 140/810
Generating nail to thread dictionary: 150/810
Generating nail to thread dictionary: 160/810
Generating nail to thread dictionary: 170/810
Generating nail to thread dictionary: 180/810
Generating nail to thread dictionary: 190/810
Generating nail to thread dictionary: 200/810
Generating nail to thread dictionary: 210/810